# Import Data

In [2]:
import pandas as pd
import numpy as np

OriginalTest = pd.read_csv("Test1.csv")
OriginalTrain = pd.read_csv("train1_modified.csv")

In [4]:
OriginalTrain.head(5)

,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Identifier,Item_Visibility_MeanRatio,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,249.8092,3735.1380,0.016047,9.30,OUT049,0.931078,14,1,0,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,48.2692,443.4228,0.019278,5.92,OUT018,0.933420,4,0,0,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,141.6180,2097.2700,0.016760,17.50,OUT049,0.960069,14,1,0,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,182.0950,732.3800,0.017834,19.20,OUT010,1.000000,15,0,0,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,53.8614,994.7052,0.009780,8.93,OUT013,1.000000,26,0,1,...,0,1,0,0,0,0,0,0,0,0


In [7]:
(OriginalTrain.isnull().sum() / len(OriginalTrain))*100

Item_Identifier              0.0
Item_MRP                     0.0
Item_Outlet_Sales            0.0
Item_Visibility              0.0
Item_Weight                  0.0
Outlet_Identifier            0.0
Item_Visibility_MeanRatio    0.0
Outlet_Years                 0.0
Item_Fat_Content_0           0.0
Item_Fat_Content_1           0.0
Item_Fat_Content_2           0.0
Outlet_Location_Type_0       0.0
Outlet_Location_Type_1       0.0
Outlet_Location_Type_2       0.0
Outlet_Size_0                0.0
Outlet_Size_1                0.0
Outlet_Size_2                0.0
Outlet_Size_3                0.0
Outlet_Type_0                0.0
Outlet_Type_1                0.0
Outlet_Type_2                0.0
Outlet_Type_3                0.0
Item_Type_Combined_0         0.0
Item_Type_Combined_1         0.0
Item_Type_Combined_2         0.0
Outlet_0                     0.0
Outlet_1                     0.0
Outlet_2                     0.0
Outlet_3                     0.0
Outlet_4                     0.0
Outlet_5  

# Preprocessing Data

In [10]:
from sklearn.preprocessing import LabelEncoder
import copy

def getEncodeData(DataFrame, Columns):
    labelEncoder = LabelEncoder()
    encodedData = copy.deepcopy(DataFrame)
    for column in Columns:
        encodedData[column] = labelEncoder.fit_transform(DataFrame[column])
    return encodedData

In [12]:
encodedData = getEncodeData(OriginalTrain, ['Item_Identifier', 'Outlet_Identifier'])
encodedData.head()

,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Identifier,Item_Visibility_MeanRatio,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,156,249.8092,3735.1380,0.016047,9.30,9,0.931078,14,1,0,...,0,0,0,0,0,0,0,0,0,1
1,8,48.2692,443.4228,0.019278,5.92,3,0.933420,4,0,0,...,0,0,0,1,0,0,0,0,0,0
2,662,141.6180,2097.2700,0.016760,17.50,9,0.960069,14,1,0,...,0,0,0,0,0,0,0,0,0,1
3,1121,182.0950,732.3800,0.017834,19.20,0,1.000000,15,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1297,53.8614,994.7052,0.009780,8.93,1,1.000000,26,0,1,...,0,1,0,0,0,0,0,0,0,0


# Split data to train and test

In [13]:
def SplitDataFrameToTrainAndTest(DataFrame, TrainDataRate, TargetAtt):
    training = DataFrame.sample(frac=TrainDataRate, random_state=1)
    testing = DataFrame.loc[~DataFrame.index.isin(training.index)]
    x_train = training.drop(TargetAtt, 1)
    y_train = training[TargetAtt]
    x_test = testing.drop(TargetAtt, 1)
    y_test = testing[TargetAtt]
    return x_train, y_train, x_test, y_test

In [14]:
x_train, y_train, x_test, y_test = SplitDataFrameToTrainAndTest(DataFrame=encodedData, TrainDataRate=0.6, TargetAtt='Item_Outlet_Sales')

# Train Data

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier

In [16]:
clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=lr)

In [21]:
clf2.fit(x_train, y_train)

C:\Users\xuant\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: Unknown label type: 'continuous'